In [13]:
from functools import reduce
import inspect
from typing import Any, Callable, Collection


type Composable = Callable[[Any], Any]


def mapper(converter: Callable) -> Callable:
    def do_map(items: Collection) -> tuple:
        result = tuple(map(converter, items))
        return result
    return do_map


def joiner(sep: str) -> Callable:
    def do_join(value: Collection) -> str:
        return sep.join(filter(bool, value))
    return do_join


def combiner(*functions: Composable) -> Composable:
    def apply(value: Any, fn: Composable) -> Any:
        return fn(value)
    return lambda val: reduce(apply, functions, val)


def func(*args, **kwargs):
    str_mapper = mapper(str)
    coma_joiner = joiner(", ")

    kwargs_handler = combiner(mapper(str_mapper), mapper(joiner("=")))
    signature_handler = combiner(mapper(coma_joiner), coma_joiner)
    
    all_args = signature_handler((str_mapper(args), kwargs_handler(kwargs.items()),),)


    print(f"{inspect.currentframe().f_code.co_name}({all_args})")


func(another_kwarg="another kwarg")
func(1)
func(10, 100, 300, test={10, 20, 30}, another_kwarg=100)
func(10, 100, 300, test=range(10), another_kwarg=300)

func(another_kwarg=another kwarg)
func(1)
func(10, 100, 300, test={10, 20, 30}, another_kwarg=100)
func(10, 100, 300, test=range(0, 10), another_kwarg=300, non_string_kwarg=[10, 11])


In [24]:
type Composable = Callable[[Any], Any]


def combiner(*functions: Composable) -> Composable:
    def apply(value: Any, fn: Composable) -> Any:
        return fn(value)
    return lambda val: reduce(apply, functions, val)


def curried(prefix, postfix):
    def curry(value):
        return f"{prefix} {value} {postfix}"
    return curry



do_stuff = combiner(
        curried(">>>", "<<<"),
        curried("!===", "===!"),
        curried("...", ",,,")
        )

do_stuff("Douglas Adams")
do_stuff

<function __main__.combiner.<locals>.<lambda>(val)>

In [5]:
from typing import Any, Callable

def curry(f: Callable[[tuple[str, str], str], str]) -> Callable[[tuple[str, str]], Any]:
    def x(a: tuple[str, str]) -> Callable[[str], str]:
        def y(b: str) -> str:
            return f(a, b)
        return y
    return x


def my_func(a:tuple[str, str], b: str) -> str:
    return b.join(a)


a = (">>>", "<<<")
b = "Douglas Adams"

curried = curry(my_func)
print(curried(a)(b))

print(my_func(a, b))

>>>Douglas Adams<<<
>>>Douglas Adams<<<


In [66]:
import pprint
import textwrap
import tabulate

prefix = "Description"
prefix_len = len("Description: ")

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam non purus eu metus maximus bibendum. Sed finibus porta porta. Mauris gravida, mauris eu dignissim volutpat, turpis sem sollicitudin nisi, at hendrerit eros mi efficitur metus."

the_dicts = [

    {"header": "Title", "text": "tabulate 0.9.0"},
    {"header": "URL", "text": "https://pypi.org/project/tabulate/"},
    {"header": prefix, "text": text},
    ]
print(tabulate.tabulate(the_dicts, tablefmt='orgtbl', colalign=('right','left'), maxcolwidths=[None, 60]))


|       Title | tabulate 0.9.0                                               |
|         URL | https://pypi.org/project/tabulate/                           |
| Description | Lorem ipsum dolor sit amet, consectetur adipiscing elit.     |
|             | Aliquam non purus eu metus maximus bibendum. Sed finibus     |
|             | porta porta. Mauris gravida, mauris eu dignissim volutpat,   |
|             | turpis sem sollicitudin nisi, at hendrerit eros mi efficitur |
|             | metus.                                                       |


In [76]:
import textwrap

prefix = "Description: "
prefix_len = len("Description: ")

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam non purus eu metus maximus bibendum. Sed finibus porta porta. Mauris gravida, mauris eu dignissim volutpat, turpis sem sollicitudin nisi, at hendrerit eros mi efficitur metus."
wrapped = textwrap.wrap(prefix+text, width=60+prefix_len, initial_indent="", subsequent_indent=" "*prefix_len, )

print(*wrapped, sep="\n")

Description: Lorem ipsum dolor sit amet, consectetur adipiscing elit.
             Aliquam non purus eu metus maximus bibendum. Sed finibus
             porta porta. Mauris gravida, mauris eu dignissim volutpat,
             turpis sem sollicitudin nisi, at hendrerit eros mi efficitur
             metus.
